IMPORT LIBRARY (SESUAI KEBUTUHAN JURNAL)

In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

np.random.seed(42)
tf.random.set_seed(42)

LOAD & PEMBERSIHAN DATASET CKD

In [ ]:
# Contoh: load dataset CSV
# Ganti path sesuai dataset Anda
data_path = "/content/drive/MyDrive/JST_TB/Dataset2/ckd-dataset-v2.csv"
data = pd.read_csv(data_path, on_bad_lines='skip')

# Clean column names: strip whitespace and remove single quotes
data.columns = data.columns.str.strip().str.replace("'", "")

# Tampilkan 5 data awal
data.head()


,bp (Diastolic),bp limit,sg,al,class,rbc,su,pc,pcc,ba,...,htn,dm,cad,appet,pe,ane,grf,stage,affected,age
0,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,...,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class,meta
2,0,0,1.019 - 1.021,1 - 1,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
3,0,0,1.009 - 1.011,< 0,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
4,0,0,1.009 - 1.011,≥ 4,ckd,1,< 0,1,0,1,...,0,0,0,1,0,0,127.281 - 152.446,s1,1,< 12


KONVERSI TARGET 'grf' (TEKS → NUMERIK)

In [ ]:
def convert_grf(value):
    if pd.isna(value):
        return np.nan

    value = str(value).strip()

    # Case: range "a - b"
    if "-" in value:
        nums = re.findall(r"\d+\.?\d*", value)
        if len(nums) == 2:
            return (float(nums[0]) + float(nums[1])) / 2

    # Case: ≥ x atau >= x
    if "≥" in value or ">=" in value:
        nums = re.findall(r"\d+\.?\d*", value)
        if len(nums) == 1:
            return float(nums[0])

    # Case: ≤ x atau <= x
    if "≤" in value or "<=" in value:
        nums = re.findall(r"\d+\.?\d*", value)
        if len(nums) == 1:
            return float(nums[0])

    return np.nan

data['grf'] = data['grf'].apply(convert_grf)

PISAHKAN INPUT (X) & TARGET (y)

In [ ]:
X = data.drop(columns=['grf'])
y = data['grf']

KONVERSI FITUR MENJADI NUMERIK

In [ ]:
X = X.apply(pd.to_numeric, errors='coerce')


IMPUTASI NILAI HILANG

In [ ]:
imputer_X = SimpleImputer(strategy='mean')
imputer_y = SimpleImputer(strategy='mean')

X = imputer_X.fit_transform(X)
y = imputer_y.fit_transform(y.values.reshape(-1, 1)).ravel()

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sg' 'al' 'class' 'su' 'bgr' 'bu' 'sod' 'sc' 'pot' 'hemo' 'pcv' 'rbcc'
 'wbcc' 'stage' 'age']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Normalisasi

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)


Split Data (70–15–15)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.30, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

print("Training:", X_train.shape)
print("Validation:", X_val.shape)
print("Testing:", X_test.shape)

Training: (141, 13)
Validation: (30, 13)
Testing: (31, 13)


FUNGSI NGUYEN–WIDROW INITIALIZATION

In [ ]:
def nguyen_widrow_init(n_input, n_hidden):
    beta = 0.7 * (n_hidden ** (1 / n_input))
    W = np.random.uniform(-0.5, 0.5, (n_input, n_hidden))

    for i in range(n_hidden):
        W[:, i] = beta * W[:, i] / np.linalg.norm(W[:, i])

    b = np.random.uniform(-beta, beta, n_hidden)
    return W, b

 BANGUN MODEL JST (SESUAI JURNAL)

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(1, activation='linear'))

# Terapkan Nguyen–Widrow pada hidden layer
W, b = nguyen_widrow_init(X_train.shape[1], 10)
model.layers[0].set_weights([W, b])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


COMPILE MODEL

In [ ]:
model.compile(
    optimizer='adam',                # Backpropagation
    loss='mean_squared_error'        # SESUAI JURNAL
)

TRAINING MODEL

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=1000,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 11852.5703 - val_loss: 10675.4131
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 11843.3438 - val_loss: 10666.6611
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 11834.2236 - val_loss: 10657.9717
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 11825.1631 - val_loss: 10649.3428
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 11816.1602 - val_loss: 10640.7793
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 11807.2207 - val_loss: 10632.2803
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 11798.3408 - val_loss: 10623.8477
Epoch 8/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 11789.5264 - val_loss: 10615.4844
Epoch 9/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 11780.7744 - val_loss: 10607.1904
Epoch 10/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 11772.0879 - val_loss: 10598.9668
Epoch 11/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 11763.466

 EVALUASI MODEL

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("MSE Testing :", mse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
MSE Testing : 3618.4084263649293


Dataset yang digunakan adalah ckd-dataset-v2.csv dengan target regresi GRF.

Model JST dibangun dengan satu hidden layer dan algoritma backpropagation sesuai jurnal rujukan.

Inisialisasi bobot menggunakan metode Nguyen–Widrow.

Fungsi evaluasi yang digunakan adalah Mean Squared Error (MSE).

Hasil pengujian menghasilkan nilai MSE sebesar 3618,41.

Proses pelatihan mencapai epoch maksimum (1000 epoch) yang menunjukkan konvergensi model berlangsung lambat.

Tingginya nilai MSE dipengaruhi oleh:

Rentang nilai GRF yang besar

Dominasi fitur diskrit dan interval

Ukuran dataset yang terbatas

Model tidak menunjukkan indikasi overfitting yang signifikan.

JST layak diterapkan pada dataset ini, namun diperlukan peningkatan kualitas data untuk memperoleh hasil yang lebih optimal.